<a href="https://colab.research.google.com/github/okwasna/dane_wszystkie/blob/main/dane_wszystkie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
pip install pandas


In [43]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score


In [44]:

# Wczytanie danych z pliku Excel
file_path = 'summary_htseq_norm2.xlsx'
df = pd.read_excel(file_path, sheet_name='summary_htseq')
# Wyświetlenie liczby wszystkich genów przed filtrowaniem
print(f"Liczba genów przed filtrowaniem: {len(df)}")

# Wyświetlenie liczby wszystkich genów przed filtrowaniem
print(f"Liczba genów przed filtrowaniem: {len(df)}")

# Obliczenie średniej i wariancji kolumny 'var' przed filtrowaniem
mean_before_filtering = df['var'].mean()
variance_before_filtering = df['var'].var()

print(f"Średnia wariancji przed filtrowaniem: {mean_before_filtering}")
print(f"Wariancja przed filtrowaniem: {variance_before_filtering}")

# Krok 1: Filtrowanie genów o niskiej wariancji
# Ustalmy próg wariancji na 10 - odrzucamy geny poniżej tej wartości
variance_threshold = 10

# Filtrowanie genów o wariancji poniżej progu
df_filtered = df[df['var'] >= variance_threshold]

# Obliczenie średniej i wariancji kolumny 'var' po filtrowaniu
mean_after_filtering = df_filtered['var'].mean()
variance_after_filtering = df_filtered['var'].var()

print(f"Średnia wariancji po filtrowaniu: {mean_after_filtering}")
print(f"Wariancja po filtrowaniu: {variance_after_filtering}")

# Zliczenie ilości genów dla lncRNA przed filtrowaniem
initial_counts = df['lncRNA'].value_counts()

# Zliczenie ilości genów dla lncRNA po filtrowaniu
filtered_counts = df_filtered['lncRNA'].value_counts()

# Obliczenie liczby usuniętych genów z lncRNA 0 i 1
removed_counts = initial_counts - filtered_counts

# Wyświetlenie liczby usuniętych genów osobno dla lncRNA = 0 i lncRNA = 1
removed_0 = removed_counts.get(0, 0)
removed_1 = removed_counts.get(1, 0)

print(f"Liczba genów usuniętych z lncRNA = 0: {removed_0}")
print(f"Liczba genów usuniętych z lncRNA = 1: {removed_1}")

# Wyświetlenie łącznej liczby genów po filtrowaniu
total_genes_after_filtering = len(df_filtered)
print(f"Łączna liczba genów po filtrowaniu: {total_genes_after_filtering}")

Liczba genów przed filtrowaniem: 28677
Liczba genów przed filtrowaniem: 28677
Średnia wariancji przed filtrowaniem: 83837021.10925484
Wariancja przed filtrowaniem: 1.4257388777828288e+20
Średnia wariancji po filtrowaniu: 108068243.92073244
Wariancja po filtrowaniu: 1.837809291273408e+20
Liczba genów usuniętych z lncRNA = 0: 3473
Liczba genów usuniętych z lncRNA = 1: 2957
Łączna liczba genów po filtrowaniu: 22247


In [53]:

# Wyświetlenie liczby próbek (kolumn) i liczby genów (wierszy) branych pod uwagę
print(f"Liczba próbek (kolumn) branych pod uwagę: {data_values.shape[1]}")
print(f"Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: {data_values.shape[0]}")


Liczba próbek (kolumn) branych pod uwagę: 33
Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: 22247


In [54]:

# Wybór kolumn zaczynających się od 'SRR7451446'
# Zakładamy, że 'SRR7451446' to nazwa kolumny, a próbki znajdują się w kolumnach
data_values = df_filtered.loc[:, 'SRR7451446':]  # Wybieramy wszystkie kolumny od 'SRR7451446'

# Wyświetlenie liczby próbek (kolumn) i liczby genów (wierszy) branych pod uwagę
print(f"Liczba próbek (kolumn) branych pod uwagę: {data_values.shape[1]}")
print(f"Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: {data_values.shape[0]}")

# Normalizacja danych (wierszy - genów)
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data_values)

# KMeans dla 40 klastrów
n_clusters = 40
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(data_normalized)

# Dodanie kolumny z wynikami KMeans do oryginalnej ramki danych
df_filtered.loc[:, f'Cluster_{n_clusters}'] = kmeans.labels_

# Obliczenie wskaźnika Silhouette
silhouette_avg = silhouette_score(data_normalized, kmeans.labels_)
print(f"Średni wskaźnik Silhouette dla {n_clusters} klastrów: {silhouette_avg:.4f}")

# Zliczenie ilości genów w każdym klastrze
cluster_counts = df_filtered[f'Cluster_{n_clusters}'].value_counts()
print(f"Ilość genów w każdym klastrze dla {n_clusters} klastrów:")
print(cluster_counts)

# Wyświetlenie całkowitej liczby genów, które zostały przypisane do klastrów
print(f"Całkowita liczba genów przypisanych do klastrów: {len(df_filtered)}")

# Zapisanie wyników do nowego pliku Excel
df_filtered.to_excel('summary_htseq_with_40_clusters.xlsx', index=False)

Liczba próbek (kolumn) branych pod uwagę: 33
Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: 22247
Średni wskaźnik Silhouette dla 40 klastrów: 0.9417
Ilość genów w każdym klastrze dla 40 klastrów:
Cluster_40
0     14768
3      2848
25     1264
10      601
18      563
11      521
6       455
14      289
19      142
26      135
9       124
33      104
30       90
29       65
28       63
38       45
2        45
37       39
22       21
27       11
39       10
36        9
32        7
12        6
20        5
35        3
16        1
4         1
13        1
23        1
8         1
24        1
31        1
1         1
5         1
34        1
17        1
21        1
15        1
7         1
Name: count, dtype: int64
Całkowita liczba genów przypisanych do klastrów: 22247


In [52]:

# Wyświetlenie liczby próbek (kolumn) i liczby genów (wierszy) branych pod uwagę
print(f"Liczba próbek (kolumn) branych pod uwagę: {data_values.shape[1]}")
print(f"Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: {data_values.shape[0]}")


Liczba próbek (kolumn) branych pod uwagę: 33
Liczba genów (wierszy) branych pod uwagę po wybraniu próbek: 22247


In [ ]:
#zrobić wizualizacje (heatmapa + wykres)

In [56]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

# Agglomerative Clustering dla 40 klastrów
agg_clustering = AgglomerativeClustering(n_clusters=40)
agg_labels = agg_clustering.fit_predict(data_normalized)

# Dodanie kolumny z wynikami Agglomerative Clustering do ramki danych
df_filtered.loc[:, 'Agg_Cluster_40'] = agg_labels

# Obliczenie wskaźnika Silhouette dla Agglomerative Clustering
agg_silhouette_avg = silhouette_score(data_normalized, agg_labels)
print(f"Średni wskaźnik Silhouette dla Agglomerative Clustering (40 klastrów): {agg_silhouette_avg:.4f}")

# Zliczenie ilości genów w każdym klastrze dla Agglomerative Clustering
agg_cluster_counts = pd.Series(agg_labels).value_counts()
print(f"Ilość genów w każdym klastrze dla Agglomerative Clustering (40 klastrów):")
print(agg_cluster_counts)


<ipython-input-56-a15933a8186c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'Agg_Cluster_40'] = agg_labels


Średni wskaźnik Silhouette dla Agglomerative Clustering (40 klastrów): 0.9416
Ilość genów w każdym klastrze dla Agglomerative Clustering (40 klastrów):
13    14768
23     2848
37     1264
12      601
14      563
30      521
28      455
24      289
32      142
6       135
11      124
17      104
16       90
2        78
15       65
25       63
38       43
5        27
10       10
0         9
18        9
1         8
8         6
3         4
9         3
7         3
4         2
34        1
33        1
26        1
29        1
36        1
31        1
35        1
20        1
22        1
39        1
27        1
19        1
21        1
Name: count, dtype: int64


In [57]:
from sklearn.cluster import DBSCAN

# DBSCAN - nie musimy podawać liczby klastrów, ale parametry epsilon i min_samples
dbscan_clustering = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan_clustering.fit_predict(data_normalized)

# Dodanie kolumny z wynikami DBSCAN do ramki danych
df_filtered.loc[:, 'DBSCAN_Cluster'] = dbscan_labels

# Zliczenie ilości genów w każdym klastrze dla DBSCAN
dbscan_cluster_counts = pd.Series(dbscan_labels).value_counts()
print(f"Ilość genów w każdym klastrze dla DBSCAN:")
print(dbscan_cluster_counts)

# Obliczenie wskaźnika Silhouette dla DBSCAN (jeśli są więcej niż 1 klaster i nie są outliery)
if len(set(dbscan_labels)) > 1 and -1 not in dbscan_labels:
    dbscan_silhouette_avg = silhouette_score(data_normalized, dbscan_labels)
    print(f"Średni wskaźnik Silhouette dla DBSCAN: {dbscan_silhouette_avg:.4f}")
else:
    print("Nie można obliczyć wskaźnika Silhouette dla DBSCAN (za mało klastrów lub obecność outlierów).")


Ilość genów w każdym klastrze dla DBSCAN:
 0     14768
 1      2847
 2      1262
 7       585
 3       558
-1       539
 4       521
 5       455
 9       250
 11      134
 13       82
 6        73
 10       52
 12       51
 14       37
 8        19
 15       14
Name: count, dtype: int64
Nie można obliczyć wskaźnika Silhouette dla DBSCAN (za mało klastrów lub obecność outlierów).


<ipython-input-57-ee26e012b907>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'DBSCAN_Cluster'] = dbscan_labels
